In [ ]:
import pandas as pd

In [ ]:
# Carregueem cada dataset en un datafrem. Cada dataset correspon a cada any, que contenen el nombre de publicacionsd dels paisos.
articles_2016 = pd.read_csv("2016-tables-country.csv", sep=',')
articles_2017 = pd.read_csv("2017-tables-country.csv", sep=',')
articles_2018 = pd.read_csv("2018-tables-country.csv", sep=',')
articles_2019 = pd.read_csv("2019-tables-country.csv", sep=',')
articles_2020 = pd.read_csv("2020-tables-country.csv", sep=',')
articles_2021 = pd.read_csv("2021-tables-country.csv", sep=',')
articles_2022 = pd.read_csv("2022-tables-country.csv", sep=',')
articles_2023 = pd.read_csv("2023-tables-country.csv", sep=',')

In [ ]:
from functools import reduce

# Llistat de tots els fitxers anteriors
dfs = [articles_2016, articles_2017, articles_2018, articles_2019, articles_2020, articles_2021, articles_2022, articles_2023 ]

# La fusió es basa en aquesta columna
merge_on = "Country/territory"

# No ens serveix la columna "#"
for df in dfs:
     if '#' in df.columns:
         df.drop('#', axis=1, inplace=True)

# Fusiona totes les taules en una sola
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=merge_on,
                                            how='outer'), dfs).fillna('void')


In [ ]:
# Eliminar valors buits
df_merged.replace('void', pd.NA, inplace=True)

# Elimina observacions que continguin valors nuls
df_merged.dropna(inplace=True)


In [ ]:
# Nomès volem les columnes que comencen per "Count"
columns_to_keep = [col for col in df_merged.columns if col.startswith('Count')]

# Filtrem el dataset amb aquestes columnes i el guardem en un nou DataFrame
df_filtered = df_merged[columns_to_keep]

df_filtered

In [ ]:
# Ara treiem la paraula "Count" i deixem nomès els anys
df_filtered.columns = df_filtered.columns.str.replace('Count ', '')

# Exporta el dataframe a un fitxer CSV
pd.DataFrame.to_csv(df_filtered, 'taules_fusionades.csv', sep=',', na_rep='.', index=False)

In [ ]:
# Creem un objecte de tipus Excel. Aquest dataset conté les puntuacions de PISA
xls = pd.ExcelFile("pisa_historic.xlsx")

# Llistat buit que contindrà els dataframes
df_list = []

# Fem una iteració per a cada fulla del fitxer Excel
for sheet_name in xls.sheet_names:
    
    # Llegeix i Guarda en un dataframe la fulla excel
    df = pd.read_excel("pisa_historic.xlsx", sheet_name=sheet_name)
    
    # La columna "year" tindrà el mateix nom que la fulla
    year = sheet_name 
    # Renombra les columnes
    df.columns = ['Country/territory', 'Mean Score'] 
    
    # Afegim la columna 'year'
    df['Year'] = year
    
    # Afegeix el dataframe a la llista
    df_list.append(df)

# Fusiona tots els DataFrames de la llista
merged_df = pd.concat(df_list, ignore_index=True)

# Volem tenir columnes separades per a cada un dels anys
pivot_df = merged_df.pivot(index='Country/territory', columns='Year', values='Mean Score')

# Fem un reset dels index
pivot_df.reset_index(inplace=True)

# Guarda el dataframe fusionat en una nova fulla dins el fitxer Excel
with pd.ExcelWriter("pisa_historic.xlsx", engine='openpyxl', mode='a') as writer:
    
    pivot_df.to_excel(writer, sheet_name='MergedData', index=False)


In [ ]:
# Llegeix el dataset que conté la informació sobre la inversió en R&D dels paisos
inversioRD = pd.read_csv("research-spending-gdp.csv", sep=',')

In [ ]:
# Convertir la columna "Year" a format numèric i eliminem valors buits
inversioRD['Year'] = pd.to_numeric(inversioRD['Year'], errors='coerce')
inversioRD = inversioRD.dropna(subset=['Year'])

# Filtrem per als anys igual o posterior al 2015
filtered_df = inversioRD[inversioRD['Year'] >= 2015]

# Exporta el dataframe a un fitxer .csv
filtered_df.to_csv('filtered_dataset.csv', index=False)

In [ ]:
# Llegeix el CSV i guarda en un DataFrame
df = pd.read_csv('merged_pub_RD.csv')

# Desfem el dataframe per tenir les columnes 'years' en files
melted_df = pd.melt(df, id_vars=['Entity', 'Code', 'Year', 'Research and development expenditure (% of GDP)'],
                    var_name='Year_Column', value_name='Publications')

# Extreu l'any de la columna 'Year_Column'
melted_df['Year_Column'] = melted_df['Year_Column'].astype(str).str.extract(r'(\d+)').astype(float)

# Fusionem aquest nou dataframe amb l'original
merged_df = pd.merge(df, melted_df[['Entity', 'Code', 'Year_Column', 'Publications']], 
                     left_on=['Entity', 'Year'], right_on=['Entity', 'Year_Column'], how='inner')

# Ja no necessitem aquesta columna
merged_df = merged_df.drop(['Year_Column'], axis=1)

# Exporta el dataset a un fitxer CSV
merged_df.to_csv('MegaDataset.csv', index=False)

In [ ]:
# Llegeix i carrega a datafraem fitxer Excel
df = pd.read_excel("pisa_historic.xlsx", engine='openpyxl')

# # Creació d'un nou DataFrame del dataset melted
melted_data = []

for index, row in df.iterrows():
    country = row['Country/territory']
    for col in df.columns[1:]:  
        year = col
        value = row[col]
        if not pd.isna(value): 
            melted_data.append({'Country/territory': country, 'Year': int(year), 'Value': value})

melted_df = pd.DataFrame(melted_data)

# Neteja la columna Value i converteix a float
melted_df['Value'] = melted_df['Value'].astype(str).str.replace(',', '.').astype(float)

# Exporta a CSV
melted_df.to_csv('PISA_cada_any.csv', index=False)

In [ ]:
# Carrega el primer dataset corresponent a R&D y Publicacions
df1 = pd.read_csv("RD_y_publicacions.csv")

# Carrega el segon dataset corresopnent a PISA cada any
df2 = pd.read_csv("PISA_cada_any.csv")

# Fusiona els datasets
merged_df = pd.merge(df1, df2, on=['Pais', 'Year'], how='inner')
merged_df['Value'] = merged_df['Value'].round(1)

# Exporta a CSV
merged_df.to_csv('fusio_pub_pisa.csv', index=False)